<a href="https://colab.research.google.com/github/VictorSM01/fcc_sms_text_classification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.7.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-01-26 12:37:47--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.05s   

2022-01-26 12:37:47 (6.56 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-01-26 12:37:47--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
# Read data
train_dataset = pd.read_csv(
                            'train-data.tsv',sep='\t',
                            header=None,
                            names=['class', 'text']
                            )
valid_dataset = pd.read_csv(
                            'valid-data.tsv',
                             sep='\t',
                             header=None,
                             names=['class', 'text']
                             )

In [ ]:
# Preprocess data
def preprocess(df):
    data = df.copy()
    data.loc[data['class'] == 'ham','class'] = 0
    data.loc[data['class'] == 'spam','class'] = 1
    labels = data.pop('class').astype(np.float32)
    data = data.values
    return data, labels

In [ ]:
train_data, train_labels = preprocess(train_dataset)
valid_data, valid_labels = preprocess(valid_dataset)

In [ ]:
# Map text features to integer sequences
encoder = keras.layers.TextVectorization()
encoder.adapt(train_data)

In [ ]:
# Create RNN model
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          523584    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 597,953
Trainable params: 597,953
Non-trai

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# Stopping condition
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                  mode="min", 
                                  patience=3,
                                  restore_best_weights=True)

history = model.fit(x=train_data,y=train_labels, epochs=15,
                    validation_data=(valid_data,valid_labels),
                    callbacks=[callback]
                    )

Epoch 1/15
131/131 [==============================] - 26s 66ms/step - loss: 0.5967 - accuracy: 0.8574 - val_loss: 0.4623 - val_accuracy: 0.8657
Epoch 2/15
131/131 [==============================] - 6s 42ms/step - loss: 0.3347 - accuracy: 0.8794 - val_loss: 0.2255 - val_accuracy: 0.9274
Epoch 3/15
131/131 [==============================] - 5s 42ms/step - loss: 0.1589 - accuracy: 0.9667 - val_loss: 0.1208 - val_accuracy: 0.9770
Epoch 4/15
131/131 [==============================] - 6s 45ms/step - loss: 0.0792 - accuracy: 0.9888 - val_loss: 0.0728 - val_accuracy: 0.9799
Epoch 5/15
131/131 [==============================] - 6s 43ms/step - loss: 0.0435 - accuracy: 0.9928 - val_loss: 0.0546 - val_accuracy: 0.9835
Epoch 6/15
131/131 [==============================] - 6s 45ms/step - loss: 0.0271 - accuracy: 0.9952 - val_loss: 0.0497 - val_accuracy: 0.9835
Epoch 7/15
131/131 [==============================] - 6s 43ms/step - loss: 0.0174 - accuracy: 0.9974 - val_loss: 0.0483 - val_accuracy: 0.982

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    
    outcome = model.predict(np.array([[pred_text]]))
    
    tag = 'ham'
    if outcome[0][0] >0.5:
        tag = 'spam'
    
    prediction = [outcome[0][0], tag]

    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.0054395096, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
